In [1]:
using DynamicPolynomials,LinearAlgebra
using JuMP
import HiGHS

# Step 1 Match Polynomials
$b*monomials=b_1*f_1 + b_2*f_2...$

In [2]:
function getLP(poly,arrayOfPolynomials,var)
    d = maxdegree(poly)
    base=monomials(var,d)
    b=coefficients(poly,base)
    c=[]
    nVars=[]
    for f in arrayOfPolynomials
        deg=d - maxdegree(f)
        push!(nVars,size(monomials(var,deg))[1])
        l = []
        for g in monomials(var,deg)
            basepoly=g*f
            push!(l,coefficients(basepoly,monomials(var,d)))
        end
        l=transpose(reduce(vcat,transpose.(l)))
        push!(c,l)
    end
    c=transpose(reduce(vcat,transpose.(c)))
    
    #Create Linear Program
    m = size(b,1) #Number of Monomials in Polynom to Match
    LP = Model(HiGHS.Optimizer) # Initialize Model
    n=size(c,2) # Number of Variables
    @variable(LP, u[1:n]>=0)
    for i in 1:m
        @constraint(LP, sum(dot(u,c[i,:])) == b[i])
    end
    print(LP)
    return (LP,u,b,c,nVars)
end

getLP (generic function with 1 method)

In [3]:
function getSolution(poly,arrayOfPolynomials,var)
    LP,u,b,c,n = getLP(poly,arrayOfPolynomials,var)
    optimize!(LP)
    sol=JuMP.value.(u)
    walk=1
    c=[]
    solPoly=[]
    for (j,i) in enumerate(n)
        coeff = view(sol,walk:walk+i-1)
        base = monomials(x,maxdegree(e)-maxdegree(F[j]))
        push!(solPoly,polynomial(coeff,base))
        push!(c,coeff)
        walk+=i
    end
    return solPoly
end

getSolution (generic function with 1 method)

## Example Functions

In [4]:
@polyvar x[1:3]
h = 3x[1] - 2x[2] - 2x[3]
f1 = 1 + 0*x[1]
f2 = x[1]-x[2]
f3 = x[1]-x[3]
f4 = x[1]^2 - 4x[2]x[3]
F=[f1,f2,f3,f4]
e=polynomial(monomials(x,1))*h

3x₁² + x₁x₂ + x₁x₃ - 2x₂² - 4x₂x₃ - 2x₃²

In [5]:
getSolution(e,F,x)

Presolving model
4 rows, 9 cols, 14 nonzeros
2 rows, 7 cols, 10 nonzeros
2 rows, 4 cols, 6 nonzeros
2 rows, 4 cols, 6 nonzeros
Presolve : Reductions: rows 2(-4); columns 4(-9); elements 6(-14)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2(5) 0s
          1     0.0000000000e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00


4-element Vector{Any}:
 0.0
 x₁ + 2.0x₂
 x₁ + 2.0x₃
 1.0